In [1]:
import numpy as np
from numpy import ndarray

from tensorflow import keras

In [2]:
from utils import NeuralNetwork, Dense, Layer, Linear, Loss, Operation, ParamOperation, Optimizer, SGDMomentum, Tanh, Trainer, Sigmoid

In [3]:
#WIP nneds more attention - stucked for two weeks and decided to move on/

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

X_train_conv, X_test_conv = X_train.reshape(-1, 1, 28, 28), X_test.reshape(-1, 1, 28, 28)


num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

In [5]:
def calculate_accuray(model, X_test, y_test):
    return np.equal(np.argmax(model.forward(X_test, inference=True), axis=1), y_test).sum() * 100.0 / X_test.shape[0]

## Convolutional operation

only simplified for 1D convolution to understand the pattern; 
for 2D convolution, please look at the comments in the code

To simplify the convolution, let's say we have an image, with the size of 5x5 pixel

$$ I = \begin{bmatrix}i_{11} & i_{12} & i_{13} & i_{14} & i_{15}\\
                      i_{21} & i_{22} & i_{23} & i_{24} & i_{25}\\
                      i_{31} & i_{32} & i_{33} & i_{34} & i_{35}\\
                      i_{41} & i_{42} & i_{43} & i_{44} & i_{45}\\
                      i_{51} & i_{52} & i_{53} & i_{54} & i_{55}\end{bmatrix}$$


and we want to calculate a new feature with a kernel of the size 3x3. Therefor we set the weights W:


$$ w = \begin{bmatrix}w_{11} & w_{12} & w_{13}\\w_{21} & w_{22} & w_{23} \\w_{31} & w_{32} & w_{33}\end{bmatrix}$$



So, we take basically the dot product between multiple 3x3 patches from the Image and the weights.


## Forward pass

In case of a 1d array of length 5 $ arr_1 = [i_1, i_2, i_3, i_4, i_5] $ and a kernel with length 3 $ k_1 = [w_1, w_2, w_3] $ we get following output: </br>


$$ o_1 = i_1 w_1 + i_2 w_2 + i_3 w_3 $$
$$ o_2 = i_2 w_1 + i_3 w_2 + i_4 w_3 $$
$$ o_3 = i_3 w_1 + i_4 w_2 + i_5 w_3 $$

The output shrinks to three elements from five in the beginning. To prevent this shrinkage, **padding** will be introduced. For this reason,  $ i_0 , i_6 = 0 $ will be introduced. So, we get:

$$ o_0 = i_0 w_1 + i_1 w_2 + i_2 w_3 $$
$$ o_4 = i_4 w_1 + i_5 w_2 + i_6 w_3 $$

with the same output size, as the input. In this case, the **stride** was 1. 

For a 2D convolution, the adaption is to pad the input in an appriopriately way.

## Backward pass

As an example, let's take $i_5$ from the output. It is used two times in this example as part of $o_3$ and $o_4$; multiplied with $w_3$ and $w_2$. So it can be written as, with an hypothetical $o_5$ and $w_1$:

$$ \frac{\partial L}{\partial t_5} = \frac{\partial L}{\partial o_4} w_3 + \frac{\partial L}{\partial o_5} w_2 + \frac{\partial L}{\partial o_6} w_1  $$


## Generalization

$$ z_{i,j,k} = b_k + \sum_{u=0}^{f_h-1} \sum_{v=0}^{f_w-1} \sum_{k'=0}^{f_{n'}-1} x_{i',j',k'} \cdot w_{u,v,k', k} \text{with}
\begin{cases}
      i' = i \times s_h + u\\
      j' = j \times s_w + v\\
 \end{cases}
 $$
 

> - z_{i,j,k} is the output if the neuron located in row i, column j and feature map k of the convolutional layer l
> - $s_h \text{and} s_w$ are vertical and horizontal strides
> - $f_h \text{and} f_w$ are height and width of the receptive field
> - $f_{n'}$ is the number of feature maps in the previous layer
> - $x_{i',j',k'}$ output of the neuron located in layer l - 1, row i', column j', feature map k'
> - $b_k$ bias term for the feature map k
> - $w_{u,v,k', k}$ weight between any neuron in feature map k of layer l and its input located at row u, column v and feature map k'

Basically, a convolutional layer gets:

- as input: [batch_size, in_channels, out_channels, img_height, img_width]
- convolves it to params with [in_channels, out_channels, param_height, param_width]
- outputs it to [batch_size, in_channels, out_channels, param_height, param_width]

## Pooling layers

Pooling layers are used to downsample a feature. This reduces the number of weights, memory usage and the computational time. There are two main pooling types:

- max_pooling, which takes the maximum out of the pooling kernel
- average_pooling, which calculates the average of the pooling kernel

Also, for me it is not clear, if pooling layers will be used in the future, since the computational "power" is increasing with gpus, tpus and this constraint seems to diminish. e.g. ResNet architecture uses pooling layers very sparsly.

## Flatten

The flatten operator transforms the output of a convolutional layer (e.g. 3D array) in a vector. 
This operator reduces the kernels to a vector, where we can feed the vector to a fully connected layer. Basically, the flatten operator allows to make predictions.

In [6]:
class Conv2D(Layer):
    
    def __init__(self, out_channels: int, param_size: int, activation: Operation = Sigmoid(),
                 flatten: bool = False) -> None:
        """requires an activation function upon initialization"""

        super().__init__(out_channels)
        
        self.activation = activation
        self.flatten = flatten
        
        self.param_size = param_size
        self.out_channels = out_channels
        

    def _setup_layer(self, input_: ndarray) -> None:
        """
        defines options for a fully connected layer
        """
        if self.seed:
            np.random.seed(self.seed)

        self.params = []

        conv_param = np.random.normal(loc=0,
                                      size=(input_.shape[1],  # input channels
                                            self.out_channels,
                                            self.param_size,
                                            self.param_size))

        self.params.append(conv_param)

        self.operations.append(Conv2D_Op(conv_param))
        self.operations.append(self.activation)
        
        if self.flatten:
            self.operations.append(Flatten())

        return None



In [7]:
class Conv2D_Op(ParamOperation):
    
    def __init__(self, weights):
        super().__init__(weights)
        
        self.param_size = weights.shape[2]
        # simplification for training purpose
        self.param_pad = self.param_size // 2

    def _pad_1d(self, input_: ndarray) -> ndarray:
        """
        pads zeros around the input based on the padding size
        """
        zeros = np.repeat(np.array([0]), self.param_pad)
        return np.concatenate([zeros, input_, zeros])

    def _pad_1d_batch(self, input_: ndarray) -> ndarray:
    
        # padding for 2D conv; each batch is treated as 1D sequence 
        output = [self._pad_1d(obs) for obs in input_]

        return np.stack(output)
        
    def _pad_2d_obs(self, input_: ndarray):

        input_pad = self._pad_1d_batch(input_)

        other = np.zeros((self.param_pad, input_.shape[0] + self.param_pad * 2))

        return np.concatenate([other, input_pad, other])
        
    def _pad_2d_channel(self, input_: ndarray):

        return np.stack([self._pad_2d_obs(channel) for channel in input_])
    
    def _get_image_patches(self, input_: ndarray):

        patches = []

        # pad the images per batch
        imgs_batch_pad = np.stack([self._pad_2d_channel(obs) for obs in input_])

        img_height = imgs_batch_pad.shape[2]

        #for each location in the image
        for h in range(img_height-self.param_size+1):
            for w in range(img_height-self.param_size+1):

                # get an image patch of the parameter size
                patch = imgs_batch_pad[:, :, h:h+self.param_size, w:w+self.param_size]
                patches.append(patch)

        return np.stack(patches)
    
    def _output(self) -> ndarray:
        """
        FORWARD PASS!
        
        1. step: Get image patches of size [batch_size, in_channels, img_height * img_width, kernel_size, kernel_size]
        2. step: reshape image patches to [batch_size, img_height * img_width, in_channels * kernel_size * kernel_size]
        3. step: reshape parameter [in_channels * kernel_size * kernel_size, out_channels]
        4. step: batch matrix multiplication [batch_size, img_height * img_width, out_channels]
        5. step: reshape to [barch_size, out_channels, img_height, img_width]
        """
        batch_size = self.input_.shape[0]

        img_size = self.input_.shape[2] * self.input_.shape[3]
        img_height = self.input_.shape[2]

        patch_size = self.param.shape[0] * self.param.shape[2] * self.param.shape[3]

        #step 1 and 2
        output_patches_reshaped = (self._get_image_patches(self.input_)
                                  .transpose(1, 0, 2, 3, 4)
                                  .reshape(batch_size, img_size, -1))

        # step 3
        param_reshaped = (self.param
                          .transpose(0, 2, 3, 1)
                          .reshape(patch_size, -1))

        # step 4 and step 5
        return (np.matmul(output_patches_reshaped, param_reshaped)
                .reshape(batch_size, img_height, img_height, -1)
                .transpose(0, 3, 1, 2))
    
    def _input_grad(self, output_grad: ndarray) -> ndarray:

        batch_size = self.input_.shape[0]

        img_size = self.input_.shape[2] * self.input_.shape[3]
        img_height = self.input_.shape[2]
        
        output_patches = (self._get_image_patches(output_grad)
                          .transpose(1, 0, 2, 3, 4)
                          .reshape(batch_size * img_size, -1))
        
        param_reshaped = (self.param
                          .reshape(self.param.shape[0], -1)
                          .transpose(1, 0))

        return (np.matmul(output_patches, param_reshaped)
                .reshape(batch_size, img_height, img_height, self.param.shape[0])
                .transpose(0, 3, 1, 2))
        
    def _param_grad(self, output_grad: ndarray) -> ndarray:
        """
        BACKWARD PASS!
        
        1. step: Get image patches of size [batch_size, in_channels, img_height * img_width, kernel_size, kernel_size]
        2. step: reshape image patches to [in_channels * param_height * param_width, batch_size * img_height * img_width]
        3. step: reshape tthe output to [batch_size * img_height * img_width, out_channels]
        4. step: batch matrix multiplication [in_channels * param_height * param_width, out_channels]
        5. step: reshape the parameter gradient to [in_channels, out_channels, param_height, param_width]
        """

        batch_size = self.input_.shape[0]

        img_size = self.input_.shape[2] * self.input_.shape[3]

        in_channels = self.param.shape[0]
        out_channels = self.param.shape[1]
        
        #step 1 and step 2 and step 3
        in_patches_reshape = (self._get_image_patches(self.input_)
                                        .reshape(batch_size * img_size, -1)
                                        .transpose(1,0))
        
        # transpose to match general pattern for backward gradient
        out_grad_reshape = output_grad.transpose(0,2,3,1).reshape(batch_size * img_size, -1)
        
        # step 4 and step 5
        return (np.matmul(in_patches_reshape, out_grad_reshape)
                        .reshape(in_channels, self.param_size, self.param_size, out_channels)
                        .transpose(0,3,1,2))

In [8]:
class Flatten(Operation):
    def __init__(self):
        super().__init__()
        
    def _output(self) -> ndarray:
        return self.input_.reshape(self.input_.shape[0], -1)
    
    def _input_grad(self, output_grad: ndarray) -> ndarray:
        return output_grad.reshape(self.input_.shape)

In [9]:
def softmax(x, axis=None):
    return np.exp(x - logsumexp(x, axis=axis, keepdims=True))

In [10]:
from scipy.special import logsumexp


#TODO: proper implementation - this is copy/paste

class SoftmaxCrossEntropy(Loss):
    
    def __init__(self, eps: float=1e-9) -> None:
        
        super().__init__()
        self.eps = eps
        
    def _output(self) -> float:
        
        if self.target.shape[1] == 0:
            raise NotImplementedError()     

        softmax_preds = softmax(self.prediction, axis=1)

        # clipping the softmax output to prevent numeric instability
        self.softmax_preds = np.clip(softmax_preds, self.eps, 1 - self.eps)

        # actual loss computation
        softmax_cross_entropy_loss = (
            -1.0 * self.target * np.log(self.softmax_preds) - \
                (1.0 - self.target) * np.log(1 - self.softmax_preds)
        )
        
        return np.sum(softmax_cross_entropy_loss) / self.prediction.shape[0]
    
    def _input_grad(self) -> ndarray:

        return (self.softmax_preds - self.target) / self.prediction.shape[0]

The loss is no beauty, but it works!!!

In [11]:
model = NeuralNetwork(
            layers=[Conv2D(out_channels=16, param_size=5, flatten=True, activation=Tanh()),
                    Dense(neurons=10,activation=Linear())],
            loss = SoftmaxCrossEntropy())

trainer = Trainer(model, SGDMomentum(lr = 0.1))

trainer.fit(X_train_conv, train_labels, X_test_conv, test_labels,
            epochs = 10,
            eval_every = 1,
            batch_size=60);

/Users/steffen/Documents/deep_learning_fundamentals/utils/operations.py:162: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-1.0 * self.input_))


Validation loss after 1 epochs is 9.986
Validation loss after 2 epochs is 8.988
Validation loss after 3 epochs is 10.053
Validation loss after 4 epochs is 7.842
Validation loss after 5 epochs is 9.555
Validation loss after 6 epochs is 9.809
Validation loss after 7 epochs is 10.407
Validation loss after 8 epochs is 9.666
Validation loss after 9 epochs is 8.497
Validation loss after 10 epochs is 8.726
